In [1]:
pip install requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import requests

API_KEY = "ZH0zsbcG7jJevOkZbWJvXbihQEFY4G7tnH8wW3U4"
response = requests.get(
    "https://secure.splitwise.com/api/v3.0/get_expenses",
    headers={"Authorization": f"Bearer {API_KEY}"}
)

if response.status_code == 200:
    print("Success! Here are your expenses:")
    print(response.json())  # Prints your Splitwise data
else:
    print("Error:", response.status_code)
    print(response.text)  # Debug errors

Success! Here are your expenses:
{'expenses': [{'id': 3744506232, 'group_id': 79916117, 'expense_bundle_id': None, 'description': 'Walmart', 'repeats': False, 'repeat_interval': None, 'email_reminder': False, 'email_reminder_in_advance': -1, 'next_repeat': None, 'details': '', 'comments_count': 1, 'payment': False, 'creation_method': 'unequal', 'transaction_method': 'offline', 'transaction_confirmed': False, 'transaction_id': None, 'transaction_status': None, 'cost': '77.21', 'currency_code': 'CAD', 'repayments': [{'from': 76291542, 'to': 35378642, 'amount': '36.6'}], 'date': '2025-04-11T01:39:42Z', 'created_at': '2025-04-11T01:39:40Z', 'created_by': {'id': 35378642, 'first_name': 'Sanjay', 'last_name': 'Kurian', 'picture': {'medium': 'https://s3.amazonaws.com/splitwise/uploads/user/default_avatars/avatar-orange4-100px.png'}, 'custom_picture': False}, 'updated_at': '2025-04-11T05:16:55Z', 'updated_by': {'id': 76291542, 'first_name': 'Aswin', 'last_name': 'Narayanan', 'picture': {'mediu

In [19]:
import pandas as pd
import requests
from datetime import datetime
from collections import defaultdict

# 1. Set up API connection
API_KEY = "Bearer ZH0zsbcG7jJevOkZbWJvXbihQEFY4G7tnH8wW3U4"
BASE_URL = "https://secure.splitwise.com/api/v3.0"

def get_groups():
    response = requests.get(
        f"{BASE_URL}/get_groups",
        headers={"Authorization": API_KEY}
    )
    return {g['id']: g['name'] for g in response.json().get('groups', [])}

def get_all_users():
    """Get all unique users from friends, current user, and expense repayments"""
    # Get friends
    friends_response = requests.get(
        f"{BASE_URL}/get_friends",
        headers={"Authorization": API_KEY}
    )
    friends = friends_response.json().get('friends', [])
    
    # Get current user
    current_user_response = requests.get(
        f"{BASE_URL}/get_current_user",
        headers={"Authorization": API_KEY}
    )
    current_user = current_user_response.json().get('user', {})
    
    # Get all expenses to find users in repayments
    expenses_response = requests.get(
        f"{BASE_URL}/get_expenses",
        headers={"Authorization": API_KEY},
        params={"limit": "2000"}
    )
    expenses = expenses_response.json().get('expenses', [])
    
    # Collect all unique users
    users = defaultdict(dict)
    
    # Add friends
    for user in friends:
        user_id = user.get('id')
        users[user_id] = {
            'name': user.get('name') or f"{user.get('first_name', '')} {user.get('last_name', '')}".strip(),
            'active': True
        }
    
    # Add current user
    if current_user:
        user_id = current_user.get('id')
        users[user_id] = {
            'name': current_user.get('name') or f"{current_user.get('first_name', '')} {current_user.get('last_name', '')}".strip(),
            'active': True
        }
    
    # Add users from repayments (mark as inactive if not already found)
    for expense in expenses:
        for repayment in expense.get('repayments', []):
            for user_type in ['to_user', 'from_user']:
                user_id = repayment.get(user_type)
                if user_id and user_id not in users:
                    users[user_id] = {
                        'name': f"User {user_id} (removed)",
                        'active': False
                    }
    
    return users

def get_expenses_dataframe():
    # Get all users
    all_users = get_all_users()
    groups = get_groups()
    
    # Get expenses
    response = requests.get(
        f"{BASE_URL}/get_expenses",
        headers={"Authorization": API_KEY},
        params={"limit": "2000"}
    )
    expenses = response.json().get('expenses', [])
    
    # Create base columns
    base_columns = ['Date', 'Group', 'Description', 'Category', 'Cost', 'Currency']
    
    # Create user columns (active users first, then inactive)
    active_users = [u['name'] for u in all_users.values() if u['active']]
    inactive_users = [u['name'] for u in all_users.values() if not u['active']]
    user_columns = active_users + inactive_users
    
    # Process each expense
    rows = []
    for expense in expenses:
        # Basic info
        row = {
            'Date': expense.get('date'),
            'Group': groups.get(expense.get('group_id'), 'No Group'),
            'Description': expense.get('description'),
            'Category': expense.get('category', {}).get('name'),
            'Cost': expense.get('cost'),
            'Currency': expense.get('currency_code')
        }
        
        # Initialize all user amounts to 0
        row.update({name: 0.0 for name in user_columns})
        
        for user_data in expense.get('users', []):
            user_info = user_data.get('user', {})
            full_name = f"{user_info.get('first_name', '')} {user_info.get('last_name', '')}".strip()
            net_balance = float(user_data.get('net_balance', 0))
        
            # Only update if this user exists in our columns
            if full_name in user_columns:
                row[full_name] = net_balance
        
        rows.append(row)
    
    # Create DataFrame
    df = pd.DataFrame(rows, columns=base_columns + user_columns)
    
    # Format date
    if not df.empty and 'Date' in df:
        df['Date'] = pd.to_datetime(df['Date']).dt.date
    
    # Format numeric columns
    numeric_cols = ['Cost'] + user_columns
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce').fillna(0.0).round(2)
    
    return df

# Run and save
try:
    df = get_expenses_dataframe()
    print(df.head())
except Exception as e:
    print(f"Error occurred: {e}")

         Date             Group Description            Category    Cost  \
0  2025-04-11   171 Anne Street     Walmart  Household supplies   77.21   
1  2025-04-11   171 Anne Street     Walmart  Household supplies  101.73   
2  2025-04-10  358 Madelaine Dr     Payment             General   29.38   
3  2025-04-10  358 Madelaine Dr     Payment             General  118.08   
4  2025-04-04  358 Madelaine Dr        Uber                Taxi   11.00   

  Currency  Aakash Raj  Preethi None  Aditya None  Rushda Najeeb  Beuju A  \
0      CAD         0.0           0.0          0.0            0.0      0.0   
1      CAD         0.0           0.0          0.0            0.0      0.0   
2      CAD         0.0           0.0          0.0            0.0      0.0   
3      CAD         0.0           0.0          0.0            0.0      0.0   
4      CAD         0.0           0.0          0.0            0.0      0.0   

   Deepak None  Aswin Narayanan  sai meghana  Praveen Kumar  Divyaa Saravanan  \
0    

In [18]:
df.to_csv("Sample.csv")

In [14]:
name_columns = df.columns.difference(['Date', 'Group', 'Description', 'Category', 'Cost', 'Currency'])

def update_description(row):
    if row['Description'] == 'Payment':
        payer = row[name_columns][row[name_columns] < 0].index.tolist()
        payee = row[name_columns][row[name_columns] > 0].index.tolist()
        
        if payer and payee:
            return f"{payee[0]} paid {payer[0]}"
    
    return row['Description']

df['Description'] = df.apply(update_description, axis=1)
def adjust_contribution(row):
    payer = row[name_columns][row[name_columns] > 0]
    payee = row[name_columns][row[name_columns] < 0]

    if len(payer) == 1:
        payer_name = payer.index[0]
        total_owed_by_others = -payee.sum()
        actual_paid = row['Cost']
        row[payer_name] = -(actual_paid - total_owed_by_others)
    return row
df = df.apply(adjust_contribution, axis=1)

In [20]:
df

,Date,Group,Description,Category,Cost,Currency,Aakash Raj,Preethi None,Aditya None,Rushda Najeeb,Beuju A,Deepak None,Aswin Narayanan,sai meghana,Praveen Kumar,Divyaa Saravanan,Vishnu Pratheep,Sanjay Kurian
0,2025-04-11,171 Anne Street,Walmart,Household supplies,77.21,CAD,0.0,0.0,0.00,0.0,0.0,0.0,-36.60,0.0,0.00,0.00,0.0,36.60
1,2025-04-11,171 Anne Street,Walmart,Household supplies,101.73,CAD,0.0,0.0,0.00,0.0,0.0,0.0,-13.61,0.0,0.00,0.00,0.0,13.61
2,2025-04-10,358 Madelaine Dr,Payment,General,29.38,CAD,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,29.38,-29.38,0.0,0.00
3,2025-04-10,358 Madelaine Dr,Payment,General,118.08,CAD,0.0,0.0,0.00,0.0,0.0,0.0,118.08,0.0,0.00,-118.08,0.0,0.00
4,2025-04-04,358 Madelaine Dr,Uber,Taxi,11.00,CAD,0.0,0.0,0.00,0.0,0.0,0.0,5.50,0.0,0.00,0.00,0.0,-5.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1290,2022-12-27,14 Ferris,Dollarama,General,8.00,CAD,0.0,0.0,4.00,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,-4.00
1291,2022-12-27,14 Ferris,Tim,General,19.78,CAD,0.0,0.0,9.89,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,-9.89
1292,2022-12-26,14 Ferris,Pickup from airport,General,50.00,CAD,0.0,0.0,50.00,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,-50.00
1293,2022-12-26,14 Ferris,Rent for jan,Rent,900.00,CAD,0.0,0.0,450.00,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,-450.00
